## Introduction to Iceberg Architecture

In [1]:
!pip3 install -r requirements.txt

  Cloning https://github.com/fastforwardlabs/cmlbootstrap to /tmp/pip-install-fn_87rg0/cmlbootstrap_da688e5f248c490bb9c0193bda96be24
  Running command git clone -q https://github.com/fastforwardlabs/cmlbootstrap /tmp/pip-install-fn_87rg0/cmlbootstrap_da688e5f248c490bb9c0193bda96be24


#### Launching a Spark Session with Iceberg

In [2]:
import cml.data_v1 as cmldata

CONNECTION_NAME = "go01-aw-dl"
conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

# Sample usage to run query through spark
EXAMPLE_SQL_QUERY = "show databases"
spark.sql(EXAMPLE_SQL_QUERY).show()

23/09/23 22:00:10 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
23/09/23 22:00:11 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/23 22:00:11 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
23/09/23 

+--------------------+
|           namespace|
+--------------------+
|         01_car_data|
|           01_car_dw|
|              adb101|
|            airlines|
|        airlines_csv|
|    airlines_iceberg|
|airlines_iceberg_...|
|      airlines_mjain|
|          airquality|
|                ajvp|
|          atlas_demo|
|            bankdemo|
|          bca_jps_l0|
|        cde_workshop|
|cde_workshop_pdef...|
|             cdedemo|
|        cdp_overview|
|      ceht_open_data|
|        ceht_scratch|
| ceht_transportation|
+--------------------+
only showing top 20 rows



In [3]:
spark.sparkContext.getConf().getAll()

23/09/23 22:00:32 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
23/09/23 22:00:32 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.


[('spark.eventLog.enabled', 'true'),
 ('spark.network.crypto.enabled', 'true'),
 ('spark.sql.hive.hwc.execution.mode', 'spark'),
 ('spark.jars',
  '/opt/spark/optional-lib/hive-warehouse-connector-assembly.jar,/opt/spark/optional-lib/iceberg-hive-runtime.jar,/opt/spark/optional-lib/iceberg-spark-runtime.jar'),
 ('spark.kerberos.renewal.credentials', 'ccache'),
 ('spark.sql.catalog.spark_catalog',
  'org.apache.iceberg.spark.SparkSessionCatalog'),
 ('spark.app.initial.jar.urls',
  'spark://100.100.175.202:42265/jars/iceberg-hive-runtime-1.1.0.1.19.7215.0-118.jar,spark://100.100.175.202:42265/jars/iceberg-spark-runtime-3.2_2.12-1.1.0.1.19.7215.0-118.jar,spark://100.100.175.202:42265/jars/hive-warehouse-connector-assembly-1.0.0.1.19.7215.0-118.jar'),
 ('spark.dynamicAllocation.maxExecutors', '49'),
 ('spark.eventLog.dir', 'file:///sparkeventlogs'),
 ('spark.hadoop.yarn.resourcemanager.principal', 'pauldefusco'),
 ('spark.kubernetes.driver.annotation.cluster-autoscaler.kubernetes.io/safe-t

### Iceberg Architecture

![alt text](../img/iceberg-metadata.png)

#### Iceberg Catalog

Iceberg comes with catalogs that enable SQL commands to manage tables and load them by name. Catalogs are configured using properties under spark.sql.catalog.(catalog_name).

In [4]:
# Show catalog and database
spark.sql("SHOW CURRENT NAMESPACE").show()

+-------------+---------+
|      catalog|namespace|
+-------------+---------+
|spark_catalog|  default|
+-------------+---------+



In [5]:
# Create a new database
#spark.sql("DROP DATABASE IF EXISTS spark_catalog.lakehouse")
spark.sql("CREATE DATABASE IF NOT EXISTS spark_catalog.lakehouse")
spark.sql("USE spark_catalog.lakehouse")

DataFrame[]

In [6]:
spark.sql("USE spark_catalog.lakehouse")

DataFrame[]

In [7]:
# Show catalog and database
spark.sql("SHOW CURRENT NAMESPACE").show()

+-------------+---------+
|      catalog|namespace|
+-------------+---------+
|spark_catalog|lakehouse|
+-------------+---------+



#### Create an Iceberg Table with Spark SQL

In [8]:
spark.sql("DROP TABLE IF EXISTS lakehouse.coffees_table PURGE")

DataFrame[]

23/09/23 22:46:09 WARN ExecutorPodsWatchSnapshotSource: Kubernetes client has been closed (this is expected if the application is shutting down.)
io.fabric8.kubernetes.client.WatcherException: too old resource version: 22174829 (22181089)
	at io.fabric8.kubernetes.client.dsl.internal.AbstractWatchManager.onStatus(AbstractWatchManager.java:265)
	at io.fabric8.kubernetes.client.dsl.internal.AbstractWatchManager.onMessage(AbstractWatchManager.java:249)
	at io.fabric8.kubernetes.client.dsl.internal.WatcherWebSocketListener.onMessage(WatcherWebSocketListener.java:93)
	at okhttp3.internal.ws.RealWebSocket.onReadMessage(RealWebSocket.java:323)
	at okhttp3.internal.ws.WebSocketReader.readMessageFrame(WebSocketReader.java:219)
	at okhttp3.internal.ws.WebSocketReader.processNextFrame(WebSocketReader.java:105)
	at okhttp3.internal.ws.RealWebSocket.loopReader(RealWebSocket.java:274)
	at okhttp3.internal.ws.RealWebSocket$2.onResponse(RealWebSocket.java:214)
	at okhttp3.RealCall$AsyncCall.execute(Re

# TEST 1 with COW

In [9]:
spark.sql("CREATE TABLE IF NOT EXISTS coffees_table (coffee_id BIGINT, coffee_size STRING, coffee_sale_ts TIMESTAMP)\
          USING ICEBERG\
          PARTITIONED BY (months(coffee_sale_ts))\
          TBLPROPERTIES ('write.delete.mode'='copy-on-write',\
                          'write.update.mode'='copy-on-write',\
                          'write.merge.mode'='copy-on-write',\
                          'format-version' = '2')")

DataFrame[]

#### Verify that a Metadata JSON file has been created under the Metadata directory

In [10]:
from CatalogUtil import CatalogUtil

In [11]:
catalogUtil = CatalogUtil("lakehouse", "coffees_table", "go01-demo", "s3")

In [12]:
print("These are now the files in the Iceberg Metadata Layer:")
catalogUtil.count_metadata_layer_files()

These are now the files in the Iceberg Metadata Layer:
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00000-be30580e-87c6-45b6-9480-2d6d8a3b47da.metadata.json
Total number of files in the Metadata Layer: 1


1

In [13]:
print("These are now the metadata files in the Iceberg Metadata Layer:")
catalogUtil.count_metadata_files()

These are now the metadata files in the Iceberg Metadata Layer:
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00000-be30580e-87c6-45b6-9480-2d6d8a3b47da.metadata.json
Total Number of Metadata Files: 1


1

In [14]:
print("These are now the manifest lists in the Iceberg Metadata Layer:")
catalogUtil.count_manifest_lists()

These are now the manifest lists in the Iceberg Metadata Layer:
Total Number of Manifest Lists: 0


0

In [15]:
print("These are now the manifest files in the Iceberg Metadata Layer:")
catalogUtil.count_manifest_files()

These are now the manifest files in the Iceberg Metadata Layer:
Total Number of Manifest Files: 0


0

In [16]:
metadata_file_path = catalogUtil.get_latest_metadata_file()

Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00000-be30580e-87c6-45b6-9480-2d6d8a3b47da.metadata.json


In [17]:
catalogUtil.print_metadata_file(spark, metadata_file_path)

/usr/local/lib/python3.7/site-packages/IPython/core/display.py:911: UserWarning: JSON expects JSONable dict or list, not JSON strings
  warnings.warn("JSON expects JSONable dict or list, not JSON strings")


<IPython.core.display.JSON object>

![alt text](../img/s3_metadata.png)

#### Notice that no snapshots or other files have been created as data has not yet been inserted.

In [18]:
spark.sql("SELECT * FROM lakehouse.coffees_table.history").show()

+---------------+-----------+---------+-------------------+
|made_current_at|snapshot_id|parent_id|is_current_ancestor|
+---------------+-----------+---------+-------------------+
+---------------+-----------+---------+-------------------+



In [19]:
spark.sql("SELECT * FROM lakehouse.coffees_table.snapshots;").show()

+------------+-----------+---------+---------+-------------+-------+
|committed_at|snapshot_id|parent_id|operation|manifest_list|summary|
+------------+-----------+---------+---------+-------------+-------+
+------------+-----------+---------+---------+-------------+-------+



In [20]:
spark.sql("SELECT * FROM lakehouse.coffees_table.files;").show()

+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+
|content|file_path|file_format|spec_id|partition|record_count|file_size_in_bytes|column_sizes|value_counts|null_value_counts|nan_value_counts|lower_bounds|upper_bounds|key_metadata|split_offsets|equality_ids|sort_order_id|
+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+
+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+



In [21]:
spark.sql("SELECT * FROM lakehouse.coffees_table.manifests;").show()

+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+
|content|path|length|partition_spec_id|added_snapshot_id|added_data_files_count|existing_data_files_count|deleted_data_files_count|added_delete_files_count|existing_delete_files_count|deleted_delete_files_count|partition_summaries|
+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+
+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+



In [22]:
spark.sql("SELECT * FROM lakehouse.coffees_table.all_data_files;").show()

+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+
|content|file_path|file_format|spec_id|partition|record_count|file_size_in_bytes|column_sizes|value_counts|null_value_counts|nan_value_counts|lower_bounds|upper_bounds|key_metadata|split_offsets|equality_ids|sort_order_id|
+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+
+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+



In [23]:
spark.sql("SELECT * FROM lakehouse.coffees_table.all_manifests;").show()

+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+---------------------+
|content|path|length|partition_spec_id|added_snapshot_id|added_data_files_count|existing_data_files_count|deleted_data_files_count|added_delete_files_count|existing_delete_files_count|deleted_delete_files_count|partition_summaries|reference_snapshot_id|
+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+---------------------+
+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+-------

### Table Insert

In [24]:
from pyspark.sql.functions import date_format

In [25]:
#Coffee_id = 1, Coffee_size = venti, coffee_sale_ts = 2023-09-01

spark.sql("INSERT INTO lakehouse.coffees_table VALUES (1, 'venti', cast(date_format('2023-09-01 10:00:00', 'yyyy-MM-dd HH:mm:ss') as timestamp))")

DataFrame[]

#### Data has been added to the data folder

In [26]:
QUERY = "select h.made_current_at,\
            s.operation,\
            h.snapshot_id,\
            h.is_current_ancestor,\
            s.summary['spark.app.id']\
        from lakehouse.coffees_table.history h\
        join lakehouse.coffees_table.snapshots s\
            on h.snapshot_id = s.snapshot_id\
            order by made_current_at;"

In [27]:
spark.sql(QUERY).toPandas()

23/09/08 21:44:57 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.


,made_current_at,operation,snapshot_id,is_current_ancestor,summary[spark.app.id]
0,2023-09-08 21:44:52.022,append,3743804437087925131,True,spark-application-1694209434520


#### Notice there are now two json files and two avro files. 

The first json file is the metadata file created when the table was created. This is the metata file prefixed by 00000. The second json file is the new metadata file reflecting the insert of one row. This is the metadata file prefixed by 00001.

In [28]:
print("These are now the files in the Iceberg Metadata Layer:")
catalogUtil.count_metadata_layer_files()

These are now the files in the Iceberg Metadata Layer:
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00000-be30580e-87c6-45b6-9480-2d6d8a3b47da.metadata.json
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00001-607ac9ea-b355-4064-b43a-321fa77bb56e.metadata.json
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/1c96feb7-51dd-47a2-b08a-0c339b086082-m0.avro
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/snap-3743804437087925131-1-1c96feb7-51dd-47a2-b08a-0c339b086082.avro
Total number of files in the Metadata Layer: 4


4

In [29]:
print("These are now the metadata files in the Iceberg Metadata Layer:")
catalogUtil.count_metadata_files()

These are now the metadata files in the Iceberg Metadata Layer:
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00000-be30580e-87c6-45b6-9480-2d6d8a3b47da.metadata.json
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00001-607ac9ea-b355-4064-b43a-321fa77bb56e.metadata.json
Total Number of Metadata Files: 2


2

In [30]:
print("These are now the manifest lists in the Iceberg Metadata Layer:")
catalogUtil.count_manifest_lists()

These are now the manifest lists in the Iceberg Metadata Layer:
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/snap-3743804437087925131-1-1c96feb7-51dd-47a2-b08a-0c339b086082.avro
Total Number of Manifest Lists: 1


1

In [31]:
print("These are now the manifest files in the Iceberg Metadata Layer:")
catalogUtil.count_manifest_files()

These are now the manifest files in the Iceberg Metadata Layer:
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/1c96feb7-51dd-47a2-b08a-0c339b086082-m0.avro
Total Number of Manifest Files: 1


1

In [32]:
metadata_file_path_list = catalogUtil.get_all_metadata_files()

Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00000-be30580e-87c6-45b6-9480-2d6d8a3b47da.metadata.json
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00001-607ac9ea-b355-4064-b43a-321fa77bb56e.metadata.json


In [33]:
catalogUtil.print_metadata_file(spark, metadata_file_path_list[0])

/usr/local/lib/python3.7/site-packages/IPython/core/display.py:911: UserWarning: JSON expects JSONable dict or list, not JSON strings
  warnings.warn("JSON expects JSONable dict or list, not JSON strings")


<IPython.core.display.JSON object>

In [34]:
catalogUtil.print_metadata_file(spark, metadata_file_path_list[1])

<IPython.core.display.JSON object>

The Avro file with the "snap" prefix is the manifest list.

In [35]:
manifest_lists_list = catalogUtil.get_all_manifest_lists()

Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/snap-3743804437087925131-1-1c96feb7-51dd-47a2-b08a-0c339b086082.avro


In [36]:
catalogUtil.print_manifest_file(spark, manifest_lists_list[0]) 

<IPython.core.display.JSON object>

The Avro file without the "snap" prefix is the manifest file.

In [37]:
manifest_files_list = catalogUtil.get_all_manifest_files()

Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/1c96feb7-51dd-47a2-b08a-0c339b086082-m0.avro


In [38]:
catalogUtil.print_manifest_file(spark, manifest_files_list[0]) 

<IPython.core.display.JSON object>

In [39]:
data_files_list = catalogUtil.get_all_data_layer_files()
# could also query the all_data_files metadata table:
# spark.sql("SELECT * FROM lakehouse.coffees_table.all_data_files;").toPandas()

Data File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/data/coffee_sale_ts_month=2023-09/00072-212-fd2d233b-96d1-4611-bbe4-677cedb22ad0-00001.parquet


In [40]:
catalogUtil.print_data_file(spark, data_files_list[0])

+---------+-----------+-------------------+
|coffee_id|coffee_size|     coffee_sale_ts|
+---------+-----------+-------------------+
|        1|      venti|2023-09-01 10:00:00|
+---------+-----------+-------------------+



### Table Merge Into

Create a staging table

In [41]:
spark.sql("DROP TABLE IF EXISTS lakehouse.coffee_staging PURGE")

DataFrame[]

In [42]:
spark.sql("CREATE TABLE IF NOT EXISTS lakehouse.coffee_staging\
            (coffee_id BIGINT, coffee_size STRING, coffee_sale_ts TIMESTAMP)\
            USING iceberg\
            PARTITIONED BY (months(coffee_sale_ts))")

DataFrame[]

In [43]:
spark.sql("INSERT INTO lakehouse.coffee_staging\
            VALUES (1, 'grande', cast(date_format('2023-07-01 11:00:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
            (2, 'grande', cast(date_format('2023-07-01 11:10:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
            (3, 'tall', cast(date_format('2023-04-01 12:01:00', 'yyyy-MM-dd HH:mm:ss') as timestamp))")

#Row: Coffee_id = 1, coffee_size = venti, coffee_sale_ts = 2023-07-01
#Row: Coffee_id = 2, coffee_size = grande, coffee_sale_ts = 2023-07-01
#Row: Coffee_id = 3, coffee_size = tall, coffee_sale_ts = 2023-04-01

DataFrame[]

Merge Into Customers Table

In [44]:
spark.sql("MERGE INTO lakehouse.coffees_table c\
            USING (SELECT * FROM lakehouse.coffee_staging) s\
            ON c.coffee_id = s.coffee_id \
            WHEN MATCHED THEN UPDATE SET c.coffee_size = s.coffee_size\
            WHEN NOT MATCHED THEN INSERT *")

23/09/08 21:45:18 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.


DataFrame[]

In [45]:
spark.sql("SELECT * FROM lakehouse.coffees_table.snapshots;").toPandas()

,committed_at,snapshot_id,parent_id,operation,manifest_list,summary
0,2023-09-08 21:44:52.022,3743804437087925131,NaN,append,s3a://go01-demo/warehouse/tablespace/external/...,{'spark.app.id': 'spark-application-1694209434...
1,2023-09-08 21:45:21.010,8285711116837308020,3.743804e+18,overwrite,s3a://go01-demo/warehouse/tablespace/external/...,"{'added-data-files': '3', 'total-equality-dele..."


In [46]:
spark.sql("SELECT * FROM lakehouse.coffees_table.manifests;").toPandas()

,content,path,length,partition_spec_id,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,added_delete_files_count,existing_delete_files_count,deleted_delete_files_count,partition_summaries
0,0,s3a://go01-demo/warehouse/tablespace/external/...,7204,0,8285711116837308020,3,0,0,0,0,0,"[(False, False, 2023-04, 2023-09)]"
1,0,s3a://go01-demo/warehouse/tablespace/external/...,7071,0,8285711116837308020,0,0,1,0,0,0,"[(False, False, 2023-09, 2023-09)]"


In [47]:
spark.sql("SELECT * FROM lakehouse.coffees_table.all_data_files;").toPandas()

,content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id
0,0,s3a://go01-demo/warehouse/tablespace/external/...,PARQUET,0,"(644,)",1,976,"{1: 33, 2: 34, 3: 39}","{1: 1, 2: 1, 3: 1}","{1: 0, 2: 0, 3: 0}",{},"{1: [1, 0, 0, 0, 0, 0, 0, 0], 2: [118, 101, 11...","{1: [1, 0, 0, 0, 0, 0, 0, 0], 2: [118, 101, 11...",None,[4],None,0
1,0,s3a://go01-demo/warehouse/tablespace/external/...,PARQUET,0,"(642,)",1,999,"{1: 39, 2: 41, 3: 39}","{1: 1, 2: 1, 3: 1}","{1: 0, 2: 0, 3: 0}",{},"{1: [2, 0, 0, 0, 0, 0, 0, 0], 2: [103, 114, 97...","{1: [2, 0, 0, 0, 0, 0, 0, 0], 2: [103, 114, 97...",None,[4],None,0
2,0,s3a://go01-demo/warehouse/tablespace/external/...,PARQUET,0,"(644,)",1,999,"{1: 39, 2: 41, 3: 39}","{1: 1, 2: 1, 3: 1}","{1: 0, 2: 0, 3: 0}",{},"{1: [1, 0, 0, 0, 0, 0, 0, 0], 2: [103, 114, 97...","{1: [1, 0, 0, 0, 0, 0, 0, 0], 2: [103, 114, 97...",None,[4],None,0
3,0,s3a://go01-demo/warehouse/tablespace/external/...,PARQUET,0,"(639,)",1,985,"{1: 39, 2: 39, 3: 39}","{1: 1, 2: 1, 3: 1}","{1: 0, 2: 0, 3: 0}",{},"{1: [3, 0, 0, 0, 0, 0, 0, 0], 2: [116, 97, 108...","{1: [3, 0, 0, 0, 0, 0, 0, 0], 2: [116, 97, 108...",None,[4],None,0


RECAP: The target table coffee was empty. We ran an insert and that created one new metadata file, one new manifest file, and one new data file.
Then we ran a Merge Into from the staging table. Of the three rows that we compared against, one was a match and two were not.
The one that was a match was rewritten to a brand new data file in the same partition (coffee_sale_ts_month=2023-09) which now has two data files.
The remaining two rows that were not a match were written to two new data files in the two respective partitions. If they had fallen under the same partition they may have been written into the same data file (next example).

In [48]:
print("These are now the files in the Iceberg Metadata Layer:")
catalogUtil.count_metadata_layer_files()

These are now the files in the Iceberg Metadata Layer:
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00000-be30580e-87c6-45b6-9480-2d6d8a3b47da.metadata.json
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00001-607ac9ea-b355-4064-b43a-321fa77bb56e.metadata.json
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00002-870aae3f-3044-4d3c-a377-acfc7d584e5e.metadata.json
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/1c96feb7-51dd-47a2-b08a-0c339b086082-m0.avro
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/46c20f3b-ab53-42fc-9f92-8bfa15aa081d-m0.avro
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/46c20f3b-ab53-42fc-9f92-8bfa15aa081d-m1.avro
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/sn

8

In [49]:
print("These are now the metadata files in the Iceberg Metadata Layer:")
catalogUtil.count_metadata_files()

These are now the metadata files in the Iceberg Metadata Layer:
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00000-be30580e-87c6-45b6-9480-2d6d8a3b47da.metadata.json
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00001-607ac9ea-b355-4064-b43a-321fa77bb56e.metadata.json
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00002-870aae3f-3044-4d3c-a377-acfc7d584e5e.metadata.json
Total Number of Metadata Files: 3


3

In [50]:
print("These are now the manifest lists in the Iceberg Metadata Layer:")
catalogUtil.count_manifest_lists()

These are now the manifest lists in the Iceberg Metadata Layer:
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/snap-3743804437087925131-1-1c96feb7-51dd-47a2-b08a-0c339b086082.avro
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/snap-8285711116837308020-1-46c20f3b-ab53-42fc-9f92-8bfa15aa081d.avro
Total Number of Manifest Lists: 2


2

In [51]:
print("These are now the manifest files in the Iceberg Metadata Layer:")
catalogUtil.count_manifest_files()

These are now the manifest files in the Iceberg Metadata Layer:
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/1c96feb7-51dd-47a2-b08a-0c339b086082-m0.avro
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/46c20f3b-ab53-42fc-9f92-8bfa15aa081d-m0.avro
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/46c20f3b-ab53-42fc-9f92-8bfa15aa081d-m1.avro
Total Number of Manifest Files: 3


3

In [52]:
metadata_file_path_list = catalogUtil.get_all_metadata_layer_files()

Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00000-be30580e-87c6-45b6-9480-2d6d8a3b47da.metadata.json
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00001-607ac9ea-b355-4064-b43a-321fa77bb56e.metadata.json
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00002-870aae3f-3044-4d3c-a377-acfc7d584e5e.metadata.json
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/1c96feb7-51dd-47a2-b08a-0c339b086082-m0.avro
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/46c20f3b-ab53-42fc-9f92-8bfa15aa081d-m0.avro
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/46c20f3b-ab53-42fc-9f92-8bfa15aa081d-m1.avro
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/snap-3743804437087925131-1-1c96feb7-51dd-47a2-b08a-0c339b

In [53]:
data_files_list = catalogUtil.get_all_data_layer_files()

Data File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/data/coffee_sale_ts_month=2023-04/00156-860-51bc1b8c-3b36-40b2-ab1c-c3956b89469f-00001.parquet
Data File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/data/coffee_sale_ts_month=2023-07/00011-833-347aaeb0-795a-4e94-9e29-c08eee837c78-00001.parquet
Data File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/data/coffee_sale_ts_month=2023-09/00072-212-fd2d233b-96d1-4611-bbe4-677cedb22ad0-00001.parquet
Data File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/data/coffee_sale_ts_month=2023-09/00072-841-14ecce1e-e231-45e0-8478-e460a3197248-00001.parquet


In [54]:
metadata_file_path_list = catalogUtil.get_all_metadata_files()

Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00000-be30580e-87c6-45b6-9480-2d6d8a3b47da.metadata.json
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00001-607ac9ea-b355-4064-b43a-321fa77bb56e.metadata.json
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/00002-870aae3f-3044-4d3c-a377-acfc7d584e5e.metadata.json


In [55]:
catalogUtil.print_metadata_file(spark, metadata_file_path_list[2])

/usr/local/lib/python3.7/site-packages/IPython/core/display.py:911: UserWarning: JSON expects JSONable dict or list, not JSON strings
  warnings.warn("JSON expects JSONable dict or list, not JSON strings")


<IPython.core.display.JSON object>

In [56]:
manifest_lists_list = catalogUtil.get_all_manifest_lists()

Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/snap-3743804437087925131-1-1c96feb7-51dd-47a2-b08a-0c339b086082.avro
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/snap-8285711116837308020-1-46c20f3b-ab53-42fc-9f92-8bfa15aa081d.avro


In [57]:
catalogUtil.print_manifest_file(spark, manifest_lists_list[1]) 

<IPython.core.display.JSON object>

In [58]:
manifest_files_list = catalogUtil.get_all_manifest_files()

Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/1c96feb7-51dd-47a2-b08a-0c339b086082-m0.avro
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/46c20f3b-ab53-42fc-9f92-8bfa15aa081d-m0.avro
Metadata File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/metadata/46c20f3b-ab53-42fc-9f92-8bfa15aa081d-m1.avro


In [59]:
catalogUtil.print_manifest_file(spark, manifest_files_list[1]) 

<IPython.core.display.JSON object>

In [60]:
catalogUtil.print_manifest_file(spark, manifest_files_list[2]) 

<IPython.core.display.JSON object>

In [61]:
data_files_list = catalogUtil.get_all_data_layer_files()

Data File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/data/coffee_sale_ts_month=2023-04/00156-860-51bc1b8c-3b36-40b2-ab1c-c3956b89469f-00001.parquet
Data File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/data/coffee_sale_ts_month=2023-07/00011-833-347aaeb0-795a-4e94-9e29-c08eee837c78-00001.parquet
Data File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/data/coffee_sale_ts_month=2023-09/00072-212-fd2d233b-96d1-4611-bbe4-677cedb22ad0-00001.parquet
Data File Path: warehouse/tablespace/external/hive/lakehouse.db/coffees_table/data/coffee_sale_ts_month=2023-09/00072-841-14ecce1e-e231-45e0-8478-e460a3197248-00001.parquet


In [62]:
catalogUtil.print_data_file(spark, data_files_list[0])

+---------+-----------+-------------------+
|coffee_id|coffee_size|     coffee_sale_ts|
+---------+-----------+-------------------+
|        3|       tall|2023-04-01 12:01:00|
+---------+-----------+-------------------+



In [63]:
catalogUtil.print_data_file(spark, data_files_list[1])

+---------+-----------+-------------------+
|coffee_id|coffee_size|     coffee_sale_ts|
+---------+-----------+-------------------+
|        2|     grande|2023-07-01 11:10:00|
+---------+-----------+-------------------+



NB: The row with coffee_id = 1 was a match between the target and staging tables. Because COW was used, a new file was written with the same row and the updated value for the "coffee_size" field.

In [64]:
print("The new data file with the updated row (size grande):")
data_files_list[2]

The new data file with the updated row (size grande):


'warehouse/tablespace/external/hive/lakehouse.db/coffees_table/data/coffee_sale_ts_month=2023-09/00072-212-fd2d233b-96d1-4611-bbe4-677cedb22ad0-00001.parquet'

In [65]:
catalogUtil.print_data_file(spark, data_files_list[2])

+---------+-----------+-------------------+
|coffee_id|coffee_size|     coffee_sale_ts|
+---------+-----------+-------------------+
|        1|      venti|2023-09-01 10:00:00|
+---------+-----------+-------------------+



In [66]:
print("The old data file with the original row (size venti):")
data_files_list[3]

The old data file with the original row (size venti):


'warehouse/tablespace/external/hive/lakehouse.db/coffees_table/data/coffee_sale_ts_month=2023-09/00072-841-14ecce1e-e231-45e0-8478-e460a3197248-00001.parquet'

In [67]:
catalogUtil.print_data_file(spark, data_files_list[3])

+---------+-----------+-------------------+
|coffee_id|coffee_size|     coffee_sale_ts|
+---------+-----------+-------------------+
|        1|     grande|2023-09-01 10:00:00|
+---------+-----------+-------------------+



In [68]:
spark.sql("SELECT * FROM lakehouse.coffees_table").show()

+---------+-----------+-------------------+
|coffee_id|coffee_size|     coffee_sale_ts|
+---------+-----------+-------------------+
|        2|     grande|2023-07-01 11:10:00|
|        1|     grande|2023-09-01 10:00:00|
|        3|       tall|2023-04-01 12:01:00|
+---------+-----------+-------------------+



23/09/08 22:31:08 WARN ExecutorPodsWatchSnapshotSource: Kubernetes client has been closed (this is expected if the application is shutting down.)
io.fabric8.kubernetes.client.WatcherException: too old resource version: 15832225 (15840015)
	at io.fabric8.kubernetes.client.dsl.internal.AbstractWatchManager.onStatus(AbstractWatchManager.java:265)
	at io.fabric8.kubernetes.client.dsl.internal.AbstractWatchManager.onMessage(AbstractWatchManager.java:249)
	at io.fabric8.kubernetes.client.dsl.internal.WatcherWebSocketListener.onMessage(WatcherWebSocketListener.java:93)
	at okhttp3.internal.ws.RealWebSocket.onReadMessage(RealWebSocket.java:323)
	at okhttp3.internal.ws.WebSocketReader.readMessageFrame(WebSocketReader.java:219)
	at okhttp3.internal.ws.WebSocketReader.processNextFrame(WebSocketReader.java:105)
	at okhttp3.internal.ws.RealWebSocket.loopReader(RealWebSocket.java:274)
	at okhttp3.internal.ws.RealWebSocket$2.onResponse(RealWebSocket.java:214)
	at okhttp3.RealCall$AsyncCall.execute(Re

Adding a partition field is a metadata operation and does not change any of the existing table data. New data will be written with the new partitioning, but existing data will remain in the old partition layout. Old data files will have null values for the new partition fields in metadata tables.

In [ ]:
print("TABLE PARTITIONS BEFORE ALTER PARTITION STATEMENT: ")
spark.sql("SELECT * FROM spark_catalog.lakehouse.p_evol_tbl.PARTITIONS").show()

In [ ]:
print("ADD PARTITION BY EVENT TIMESTAMP MONTHS: ")
print("ALTER TABLE spark_catalog.lakehouse.p_evol_tbl ADD PARTITION FIELD months(event_ts)")
spark.sql("ALTER TABLE spark_catalog.lakehouse.p_evol_tbl ADD PARTITION FIELD months(event_ts)")
#spark.sql("ALTER TABLE spark_catalog.lakehouse.part_evol_tbl REPLACE PARTITION FIELD hours(dob) WITH state")
#spark.sql("ALTER TABLE prod.db.sample ADD PARTITION FIELD month")

#ALTER TABLE spark_catalog.lakehouse.part_evol_tbl ADD PARTITION FIELD days(event_ts)

In [ ]:
print("TABLE PARTITIONS AFTER ALTER PARTITION STATEMENT: ")
spark.sql("SELECT * FROM spark_catalog.lakehouse.p_evol_tbl.PARTITIONS").show()

In [ ]:
appendDf = dfTestData.sample(fraction=0.3, seed=3)

In [ ]:
appendDf.dtypes

In [ ]:
appendDf.rdd.getNumPartitions()

In [ ]:
appendDf.show()

In [ ]:
appendDf.sortWithinPartitions("country").show()

In [ ]:
#appendDf.sortWithinPartitions("country", "month(event_ts)").show()

In [ ]:
appendDf.sortWithinPartitions("country").writeTo("spark_catalog.lakehouse.p_evol_tbl").using("iceberg").append() #.append()#replace()#overwritePartitions()#create()

In [ ]:
print("TABLE PARTITIONS AFTER APPEND: ")
spark.sql("SELECT * FROM spark_catalog.lakehouse.p_evol_tbl.PARTITIONS").show(100)

Dropping a partition field is a metadata operation and does not change any of the existing table data. New data will be written with the new partitioning, but existing data will remain in the old partition layout.

In [ ]:
spark.sql("ALTER TABLE spark_catalog.lakehouse.part_evol_tbl DROP PARTITION FIELD bucket(16, device_id)")

In [ ]:
print("TABLE PARTITIONS AFTER ALTER PARTITION STATEMENT: ")
spark.sql("SELECT * FROM spark_catalog.lakehouse.part_evol_tbl.PARTITIONS").show()

##### Only json files have been added (one per each time you repartitioned) but Avro files have stayed the same

In [ ]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket("go01-demo")

metadata_file_list = []

print("Current Metadata Files: \n")

for object_summary in my_bucket.objects.filter(Prefix=metadata_path+"/metadata"):
    #print(object_summary.key +"\n")
    metadata_file_list.append(object_summary.key)
    
metadata_file_list

In [ ]:
spark.sql("CREATE TABLE IF NOT EXISTS customer_table (id BIGINT, state STRING, country STRING, dob TIMESTAMP) USING iceberg PARTITIONED BY ( hours(dob))")

In [ ]:
spark.sql("SELECT HOUR(dob) FROM spark_catalog.lakehouse.customer_table").show()

In [ ]:
spark.sql("SELECT DAY(dob) FROM spark_catalog.lakehouse.customer_table").show()

### Dropping Tables

In [ ]:
spark.sql("DROP TABLE IF EXISTS lakehouse.staging")

Validate that the metadata folder is now empty but the data folder still retains parquet files.

![alt text](../img/s3_droptable_1.png)

![alt text](../img/s3_droptable_2.png)

![alt text](../img/s3_droptable_3.png)

In [ ]:
spark.sql("ALTER TABLE lakehouse.customers_table\
            SET TBLPROPERTIES ('format-version' = '2')")

In [ ]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket("go01-demo")

metadata_file_list = []

for object_summary in my_bucket.objects.filter(Prefix=metadata_path):
    print(object_summary.key +"\n")
    metadata_file_list.append(object_summary.key)

In [ ]:
print("Showing " + metadata_file_list[3])
spark.read.option("multiline","true").json("s3a://go01-demo/" + metadata_file_list[3]).toPandas()